## Setup

In [1]:
DEVICE = 'cuda:0' #change as needed

In [2]:
from collections.abc import Callable
from typing import Union

In [3]:
from functools import partial
import gc

In [4]:
import torch

In [5]:
import torch.nn.functional as F

In [6]:
from transformer_lens import HookedTransformer

In [7]:
from neuron_choice import neuron_choice
from entropy.entropy_intervention import make_hooks, store_conditioning_hook
from argparse import Namespace
from utils import NAME_TO_COMBO

## Constants that define the experiment

In [8]:
INTERVENTION_TYPE = "zero_ablation"

## Input text and model

In [9]:
prompt = "Yesterday (21 December) the Government announced a package of support for hospitality and leisure businesses that are losing trade because of the O"

In [10]:
model = HookedTransformer.from_pretrained('allenai/OLMo-7B-0424-hf', refactor_glu=True, device=DEVICE)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Loaded pretrained model allenai/OLMo-7B-0424-hf into HookedTransformer


In [11]:
my_input_ids = model.to_tokens(prompt)
str_tokens = model.to_str_tokens(my_input_ids)
print(str_tokens)

['<|endoftext|>', 'Yesterday', ' (', '21', ' December', ')', ' the', ' Government', ' announced', ' a', ' package', ' of', ' support', ' for', ' hospitality', ' and', ' leisure', ' businesses', ' that', ' are', ' losing', ' trade', ' because', ' of', ' the', ' O']


## List of weakening neurons

In [12]:
args = Namespace(
    work_dir='.', wcos_dir='.',
    model='allenai/OLMo-7B-0424-hf',
    neuron_subset_name="weakening",
    gate='-', post='+',
    activation_location='mlp.hook_post',
    intervention_type=INTERVENTION_TYPE,
)

In [13]:
neuron_list = neuron_choice(
            args,
            category_key=NAME_TO_COMBO[args.neuron_subset_name],
            subset=243,
            baseline=False
        )
#print(neuron_list)

In [14]:
neuron_list = [(int(lix), int(nix)) for lix,nix in neuron_list]

In [15]:
short_neuron_list = [(int(lix), int(nix)) for lix,nix in neuron_list if lix==model.cfg.n_layers-1]

In [16]:
print("total number of neurons considered:", len(neuron_list))
print("number of final layer neurons considered:", len(short_neuron_list))

total number of neurons considered: 243
number of final layer neurons considered: 58


## Functions

In [17]:
def store_activation_hook(activation, hook, cache_dict, neuron=None):
    cache_dict[(hook.name, neuron)] = activation[...,neuron].clone()

In [18]:
def make_all_saving_hooks(layers: list[int]|int|None=None):
    if layers is None:
        layers = list(range(model.cfg.n_layers))
    elif isinstance(layers, int):
        layers = [layers]
    hook_list = []
    stored_values = {}
    for lix, nix in neuron_list:
        if layers is not None and lix not in layers:
            continue
        hook_list.append((
            f'blocks.{lix}.mlp.hook_post',
            partial(store_activation_hook,
                    cache_dict=stored_values, neuron=nix,
            )
        ))
    return hook_list, stored_values

In [37]:
def make_all_ablation_hooks(layers: list[int]|int|None=None):
    if layers is None:
        layers = list(range(model.cfg.n_layers))
    elif isinstance(layers, int):
        layers = [layers]
    hook_list = []
    conditioning_values = {}
    for lix, nix in neuron_list:
        if layers is not None and lix not in layers:
            continue
        conditioning_values[(lix,nix)]={}#this is possible because one list is a subset of the other
        new_hooks = make_hooks(
            args,
            lix, nix,
            conditioning_value=conditioning_values[(lix,nix)],
            sign=(model.blocks[lix].mlp.W_gate[...,nix]@model.blocks[lix].mlp.W_in[...,nix]).item(),
            mean_value=0.0,
        )
        hook_list.extend(new_hooks)
    return hook_list, conditioning_values

In [20]:
def run_and_clear(model=model, input_ids=my_input_ids, fwd_hooks=[], conditioning_values={}, clear_conditions=True):
    logits = model.run_with_hooks(input_ids, fwd_hooks=fwd_hooks)
    if clear_conditions:
        conditioning_values = {key:{} for key in conditioning_values.keys()}
    return logits, conditioning_values

In [21]:
def logits_to_entropy(logit_tensor: torch.Tensor) -> torch.Tensor:
    """Compute entropy from logits. Ignore any attention mask problems (not relevant in this notebook).

    Args:
        logit_tensor (torch.Tensor): tensor of logits, shape (batch, pos, vocab)

    Returns:
        torch.Tensor: tensor of entropies, shape (batch, pos)
    """
    probs = F.softmax(logit_tensor, dim=-1)
    entropies = - torch.sum(probs * torch.log(probs + 1e-8), dim=-1)
    return entropies

In [22]:
def get_entropy_value(layers=None, input_ids=my_input_ids, clear_logits=True, clear_conditions=True, save_activations=False):
    save_hooks, save_values = [], {}
    if save_activations:
        save_hooks, save_values = make_all_saving_hooks()
    conditioning_hooks, conditioning_values = make_all_ablation_hooks(layers=layers)
    hook_list = [*save_hooks, *conditioning_hooks]
    logits, conditioning_values = run_and_clear(input_ids=input_ids, fwd_hooks=hook_list, conditioning_values=conditioning_values, clear_conditions=clear_conditions)
    entropy = logits_to_entropy(logits)[:,-1].item()
    if clear_logits:
        del hook_list
        logits=None
        gc.collect()
        torch.cuda.empty_cache()
    return logits, entropy, save_values, conditioning_values

## The effect to explain

In [23]:
logits_clean, entropy_clean, activations_clean, conditions_clean = get_entropy_value(layers=[], clear_logits=False, clear_conditions=False, save_activations=True)
print("clean entropy:", entropy_clean)

clean entropy: 0.011461791582405567


In [24]:
# del logits_all_ablated, entropy_all_ablated, activations_all_ablated, conditions_all_ablated
# gc.collect()
# torch.cuda.empty_cache()

In [25]:
logits_all_ablated, entropy_all_ablated, activations_all_ablated, conditions_all_ablated = get_entropy_value(clear_logits=False, clear_conditions=False, save_activations=True)
print("entropy when ablating all weakening neurons if gate-_post+ is fulfilled:", entropy_all_ablated)

entropy when ablating all weakening neurons if gate-_post+ is fulfilled: 9.571281433105469


In [26]:
activations_clean

{('blocks.0.mlp.hook_post',
  3467): tensor([[ 0.0530,  0.1673,  0.0095, -0.0179,  0.0475, -0.0008,  0.0026,  0.0860,
           0.0521,  0.0149,  0.0193, -0.0030,  0.0083, -0.0131,  0.0023, -0.0110,
           0.0867, -0.0079, -0.0162, -0.0108,  0.0184, -0.0599, -0.0148, -0.0284,
           0.0118,  0.2819]], device='cuda:0', grad_fn=<CloneBackward0>),
 ('blocks.0.mlp.hook_post',
  365): tensor([[ 2.2283e-05,  4.7199e-02,  6.3295e-02, -9.2658e-03, -2.6876e-02,
           3.2910e-03,  2.4767e-03,  9.3331e-03,  4.9250e-01,  6.3365e-03,
           9.1386e-02,  1.0032e-02,  2.2927e-02,  3.9378e-03,  3.6545e-03,
           3.5673e-04,  1.2086e-02,  8.1528e-03,  4.8906e-04,  3.2970e-03,
           6.6639e-02, -4.8478e-02,  2.2764e-03, -2.2786e-04,  1.5206e-03,
          -3.2523e-02]], device='cuda:0', grad_fn=<CloneBackward0>),
 ('blocks.1.mlp.hook_post',
  7896): tensor([[-0.0209,  0.0025,  0.0770,  0.0525,  0.0361,  0.0829,  0.0071,  0.0607,
           0.3042, -0.0098,  0.0656,  0.0254,  

In [27]:
conditions_all_ablated

{(12,
  6572): {'blocks.12.mlp.hook_pre': tensor([[-0.4255,  0.5205,  0.3171,  0.5524,  0.5154,  1.0460,  0.6049, -0.4128,
           -2.3889, -0.0734,  0.4984,  0.4275, -0.1948, -0.6114,  0.1196,  0.1333,
           -0.0318,  0.2690,  0.3353,  0.4310, -1.1977,  0.0562,  0.5959,  0.3711,
            0.2991,  0.5511]], device='cuda:0', grad_fn=<CloneBackward0>), 'blocks.12.mlp.hook_post': tensor([[-0.1043,  0.1278,  0.0328,  0.2909, -0.0748,  0.0751,  0.0475, -0.0327,
            0.2339, -0.0084,  0.0364,  0.0372,  0.0308,  0.1271, -0.0029,  0.0020,
            0.0056, -0.0034,  0.0298,  0.0670,  0.2056, -0.0024, -0.0060,  0.0109,
            0.0367,  0.0894]], device='cuda:0', grad_fn=<CloneBackward0>)},
 (17,
  8074): {'blocks.17.mlp.hook_pre': tensor([[ 3.1504, -3.4058, -3.3679, -3.0148, -2.9656, -3.2254, -3.8686, -3.2039,
           -3.7395, -4.1307, -3.2179, -3.2600, -3.4974, -3.4569, -2.8174, -3.7038,
           -3.8342, -3.4112, -3.5458, -3.0091, -2.5354, -3.2538, -3.2790, -3.248

## Direct path

We look at the total output of the relevant neurons. How does this vector translate into logits? What is the entropy of clean logits plus these logits?

In [31]:
relevant_activations = {
    (int(key[0].split('.')[1]), key[1]):value[0,-1]
    for key,value in activations_clean.items()
    if conditions_all_ablated[(int(key[0].split('.')[1]), key[1])][key[0].replace('post', 'pre')][0,-1]<0
    and conditions_all_ablated[(int(key[0].split('.')[1]), key[1])][key[0]][0,-1]>0
}
print(len(relevant_activations))

41


In [32]:
relevant_activations.keys()

dict_keys([(0, 3467), (2, 3972), (4, 7010), (5, 2990), (10, 10026), (11, 2077), (12, 6782), (12, 6695), (15, 446), (16, 8880), (16, 7084), (17, 8074), (18, 6799), (18, 8088), (19, 9428), (19, 4644), (22, 10594), (23, 13), (23, 9611), (24, 9659), (24, 8005), (25, 4595), (25, 6139), (26, 8816), (26, 6841), (26, 202), (28, 5325), (28, 4411), (28, 8092), (29, 7261), (29, 6527), (29, 2271), (29, 4131), (30, 1508), (30, 9996), (30, 6219), (31, 3498), (31, 9490), (31, 2604), (31, 8845), (31, 4568)])

In [35]:
direct_output = torch.sum(torch.stack([
    activation * model.blocks[layer].mlp.W_out[neuron]
    for (layer,neuron), activation in relevant_activations.items()
]), dim=0)
direct_output.shape

torch.Size([4096])

In [36]:
direct_output_logits = direct_output@model.W_U

In [37]:
direct_output_logits.shape

torch.Size([50304])

In [38]:
top = torch.topk(direct_output_logits, k=16)
bottom = torch.topk(direct_output_logits, k=16, largest=False)

In [39]:
print(top.values)
print(model.to_str_tokens(top.indices))

tensor([1.4494, 1.3602, 1.3591, 1.3082, 1.2905, 1.2548, 1.1833, 1.1735, 1.1469,
        1.1364, 1.1320, 1.1232, 1.1111, 1.0872, 1.0818, 1.0567],
       device='cuda:0', grad_fn=<TopkBackward0>)
[',', ' and', '-', ' in', ' (', '.', ' a', ' the', '\n', ' on', ' to', ' for', ' as', ' at', '/', ' or']


In [40]:
print(bottom.values)
print(model.to_str_tokens(bottom.indices))

tensor([-0.3242, -0.3121, -0.3015, -0.2955, -0.2925, -0.2920, -0.2917, -0.2887,
        -0.2883, -0.2882, -0.2879, -0.2870, -0.2862, -0.2859, -0.2852, -0.2849],
       device='cuda:0', grad_fn=<TopkBackward0>)
['力', '�', 'ическ', '�', 'ист', '�', '�', 'っている', '氏', 'лем', 'ной', '息', '民', '気', 'った', '政']


In [44]:
clean_minus_direct_logits = logits_clean[:,-1] - direct_output_logits
clean_minus_direct_entropy = logits_to_entropy(clean_minus_direct_logits).item()
print(clean_minus_direct_entropy)

0.011677267029881477


Conclusion: the direct effect explains nothing, so it's the indirect effect!
Next question: indirect effect via earlier tokens, or just on the final token?

## Ablating weakening neurons generally

In [ ]:
args.gate=None
args.post=None

hooks_general = []
stored_values = {}
for layer_list, nix in neuron_list:
    stored_values[(layer_list,nix)]={}
    hooks_general += make_hooks(
        args,
        layer_list, nix,
        conditioning_value=stored_values[(layer_list,nix)],
        sign=(model.W_gate[layer_list,:,nix]@model.W_in[layer_list,:,nix]).item(),
        mean_value=0.0,
    )

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.38 GiB. GPU 6 has a total capacity of 47.40 GiB of which 2.41 GiB is free. Including non-PyTorch memory, this process has 44.98 GiB memory in use. Of the allocated memory 41.08 GiB is allocated by PyTorch, and 3.60 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Is there a single neuron responsible for this?

We're looking for a weakening neuron whose $w_\text{out}$ corresponds as closely as possible to the token 'mic'.

In [22]:
mic = model.W_U[:,model.to_single_token('mic')]

In [25]:
scores = []
for l,n in neuron_list:
    wout = model.W_out[l,n]
    wout /= torch.norm(wout)
    scores.append(wout@mic)
scores = torch.tensor(scores)
print(scores)

tensor([-5.5654e-03, -1.9865e-02,  2.3740e-03,  2.3131e-03, -1.0920e-02,
        -1.1843e-02, -5.0423e-03,  6.3631e-05,  6.2427e-03, -2.7632e-03,
        -1.7804e-02, -3.3106e-03, -2.4462e-02,  1.4472e-02,  3.5042e-03,
         1.0408e-03, -6.3770e-03,  1.5915e-02, -7.3378e-03,  4.2016e-03,
        -2.5157e-02, -1.2029e-02, -4.9860e-04,  2.5065e-03,  1.2379e-02,
        -1.4918e-02, -3.8517e-03,  6.9253e-04,  1.1168e-02, -1.2686e-02,
        -2.5985e-02, -1.1120e-02,  6.2533e-02, -4.0599e-04,  4.4302e-03,
        -7.2232e-05,  2.3728e-02, -7.4714e-03, -2.7623e-03, -9.6072e-03,
        -7.5862e-03, -3.2695e-02, -1.2145e-03, -1.2980e-02, -6.6941e-04,
         3.1989e-02, -3.7805e-03,  1.3651e-02, -6.6925e-03, -8.9840e-04,
        -2.2595e-02,  7.9196e-03,  1.5796e-02, -8.6340e-03, -5.0332e-04,
         3.4942e-04, -1.7101e-02,  8.4782e-03, -2.1459e-03, -1.4602e-02,
         4.4703e-03,  4.5846e-03,  5.6527e-03,  1.0334e-02, -2.8266e-03,
         1.0407e-02, -1.4816e-02,  1.0181e-03, -2.0

In [27]:
torch.topk(scores, k=16)

torch.return_types.topk(
values=tensor([0.1670, 0.1110, 0.0625, 0.0482, 0.0419, 0.0352, 0.0351, 0.0328, 0.0327,
        0.0327, 0.0321, 0.0320, 0.0260, 0.0249, 0.0246, 0.0237]),
indices=tensor([ 80, 168,  32, 204, 148,  72,  75, 173, 141, 155, 138,  45, 110,  95,
        206,  36]))

In [37]:
torch.topk(scores, k=16, largest=False)

torch.return_types.topk(
values=tensor([-0.0452, -0.0336, -0.0327, -0.0296, -0.0275, -0.0271, -0.0260, -0.0255,
        -0.0252, -0.0246, -0.0245, -0.0226, -0.0211, -0.0209, -0.0206, -0.0199]),
indices=tensor([105, 111,  41,  96, 215, 187,  30, 182,  20, 135,  12,  50, 183, 159,
        242,   1]))

In [28]:
neuron_list[80]

(tensor(31, device='cuda:0'), tensor(3498, device='cuda:0'))

In [34]:
values, indices = torch.topk(model.blocks[31].mlp.W_out[3498,:]@model.W_U, k=16)
for j in range(16):
    print((model.to_single_str_token(indices[j].item()), values[j]))

('re', tensor(0.1918, device='cuda:6'))
('b', tensor(0.1895, device='cuda:6'))
('c', tensor(0.1863, device='cuda:6'))
('w', tensor(0.1831, device='cuda:6'))
('p', tensor(0.1822, device='cuda:6'))
('g', tensor(0.1808, device='cuda:6'))
('to', tensor(0.1789, device='cuda:6'))
('f', tensor(0.1776, device='cuda:6'))
('m', tensor(0.1770, device='cuda:6'))
('st', tensor(0.1760, device='cuda:6'))
('sh', tensor(0.1730, device='cuda:6'))
('de', tensor(0.1711, device='cuda:6'))
('d', tensor(0.1711, device='cuda:6'))
('t', tensor(0.1700, device='cuda:6'))
('h', tensor(0.1699, device='cuda:6'))
('se', tensor(0.1661, device='cuda:6'))


In [36]:
ranks = []
for i,(l,n) in enumerate(neuron_list):
    wout = model.blocks[l].mlp.W_out[n,:]
    wout /= torch.norm(wout)
    all_scores = wout@model.W_U
    ranks.append(torch.count_nonzero(all_scores>scores[i])-1)
ranks = torch.tensor(ranks)
print(ranks)
print(torch.topk(ranks, k=16))

tensor([33027, 44794, 11722, 17507, 46631, 25442, 34436, 23220, 11846, 34613,
        46425, 25834, 46371,  5025, 15681, 22796, 31459,  8902, 36383, 11759,
        49177, 37462, 25061, 18214,  8414, 38222, 34092, 20984,  8966, 43250,
        48547, 35267,  1817, 25002, 11094, 23719,   204, 22263, 34918, 49288,
        43525, 32577, 27633, 26429, 24108,  8951, 36769,  5448, 34544, 22052,
        49540,  4933,  5594, 40541, 23993, 21516, 40685,  9317, 31069, 45143,
        16945, 13908,  9458,  8681, 33606,  7666, 45361, 18988, 28537, 36553,
         3530, 27726,  5330,  7637,  2332,  3959, 42244, 21798, 44665, 28791,
          767, 43145, 14870, 10781, 29001, 45069, 48040, 27130, 45341,  3937,
         4669, 12602, 35360,  3953, 34059,  3986, 48969,  2957, 25602, 13362,
        47640,  1403, 12754, 20264, 25564, 31879, 29734, 16825, 36359, 44763,
          360, 49651, 14619, 26784, 24379, 10822, 17850, 35104, 13581, 34743,
        43165,  1346, 27405,  4292,  8312, 15292, 32876, 12042, 

In [38]:
torch.topk(ranks, k=16, largest=False)

torch.return_types.topk(
values=tensor([ 204,  360,  739,  767, 1142, 1346, 1403, 1817, 2200, 2332, 2957, 3006,
        3262, 3340, 3530, 3931]),
indices=tensor([ 36, 110, 130,  80, 144, 121, 101,  32, 168,  74,  97, 195, 164, 189,
         70, 173]))

In [39]:
import gc

In [40]:
gc.collect()
torch.cuda.empty_cache()

## Analysing model hidden states on the example

In [46]:
for layer in range(32):
    for sublayer in ['mid', 'post']:
        print('===========================')
        print(f'Layer {layer}, {sublayer}:')
        top_token_vi = torch.topk(cache[f'blocks.{layer}.hook_resid_{sublayer}'][0,25]@model.W_U, k=4)
        for j in range(4):
            print((model.to_str_tokens(top_token_vi.indices[j]), top_token_vi.values[j]))

Layer 0, mid:
([' in'], tensor(0.2816, device='cuda:6'))
([' of'], tensor(0.2600, device='cuda:6'))
(['\n'], tensor(0.2556, device='cuda:6'))
([' then'], tensor(0.2409, device='cuda:6'))
Layer 0, post:
(["'"], tensor(0.3902, device='cuda:6'))
(['-'], tensor(0.3567, device='cuda:6'))
(['ops'], tensor(0.3376, device='cuda:6'))
(['xygen'], tensor(0.3136, device='cuda:6'))
Layer 1, mid:
(['xygen'], tensor(0.3256, device='cuda:6'))
(["'"], tensor(0.3223, device='cuda:6'))
(['ops'], tensor(0.3137, device='cuda:6'))
(['ats'], tensor(0.3126, device='cuda:6'))
Layer 1, post:
(['xygen'], tensor(0.3145, device='cuda:6'))
(['ats'], tensor(0.2860, device='cuda:6'))
(['asis'], tensor(0.2826, device='cuda:6'))
(['ops'], tensor(0.2819, device='cuda:6'))
Layer 2, mid:
(['xygen'], tensor(0.3330, device='cuda:6'))
(['cean'], tensor(0.3086, device='cuda:6'))
(['asis'], tensor(0.2923, device='cuda:6'))
(['ceans'], tensor(0.2916, device='cuda:6'))
Layer 2, post:
(['xygen'], tensor(0.3385, device='cuda:6'))


In [47]:
for layer in range(32):
    for sublayer in ['mid', 'post']:
        print(f'Layer {layer}, {sublayer}: {cache[f'blocks.{layer}.hook_resid_{sublayer}'][0,25]@mic}')

Layer 0, mid: -0.009818505495786667
Layer 0, post: 0.1756778061389923
Layer 1, mid: 0.1873852014541626
Layer 1, post: 0.16365423798561096
Layer 2, mid: 0.18060725927352905
Layer 2, post: 0.17816656827926636
Layer 3, mid: 0.16314618289470673
Layer 3, post: 0.19512557983398438
Layer 4, mid: 0.18644800782203674
Layer 4, post: 0.20833870768547058
Layer 5, mid: 0.16482438147068024
Layer 5, post: 0.15313085913658142
Layer 6, mid: 0.14475101232528687
Layer 6, post: 0.22487929463386536
Layer 7, mid: 0.22021272778511047
Layer 7, post: 0.2116979956626892
Layer 8, mid: 0.19317162036895752
Layer 8, post: 0.2249262034893036
Layer 9, mid: 0.2260773777961731
Layer 9, post: 0.187677800655365
Layer 10, mid: 0.1936216801404953
Layer 10, post: 0.2052566558122635
Layer 11, mid: 0.22408485412597656
Layer 11, post: 0.19123020768165588
Layer 12, mid: 0.21115386486053467
Layer 12, post: 0.23132723569869995
Layer 13, mid: 0.2442898154258728
Layer 13, post: 0.25902628898620605
Layer 14, mid: 0.27178335189819336